## Import Library yang Dibutuhkan

In [1]:
import os
import time
import random
import psutil
import numpy as np
import librosa
import pandas as pd
import matplotlib.pyplot as plt
import threading
import gc


from scipy.signal import hilbert
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier

import tensorflow as tf
from tensorflow.keras import layers, models, regularizers, optimizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

## Konfigurasi Seeds

Konfigurasi seeds penting untuk reproducability

In [2]:
SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
np.random.seed(SEED)
random.seed(SEED)

## Pembuatan Kelas Utilitas

### Kelas untuk Menghitung Waktu

In [3]:
class Timer:
    """Simple timer utility to measure elapsed wall-clock 
    time (in seconds)."""
    def __init__(self):
        self.start_time = None
        self.elapsed = 0.0

    def start(self):
        self.start_time = time.perf_counter()

    def stop(self):
        if self.start_time is not None:
            self.elapsed += time.perf_counter() - self.start_time
            self.start_time = None
        return self.elapsed

    def reset(self):
        self.start_time = None
        self.elapsed = 0.0

    def get_elapsed(self):
        return self.elapsed

### Kelas mengukur memori

In [4]:
class MemoryTracker:
    """
    Peak RAM tracker berbasis Peak RSS (MB).
    - baseline: RSS tepat saat mulai tracking
    - peak: RSS maksimum selama tracking (sampling tiap interval)
    - delta_peak_mb = peak - baseline  (untuk klaim 'extra memory')
    """
    def __init__(self, interval=0.02):
        self.process = psutil.Process(os.getpid())
        self.interval = float(interval)
        self.baseline_bytes = None
        self.peak_bytes = None
        self._stop = False
        self._thread = None

    def _rss_bytes(self):
        return self.process.memory_info().rss  # bytes

    def _watch(self):
        while not self._stop:
            cur = self._rss_bytes()
            if cur > self.peak_bytes:
                self.peak_bytes = cur
            time.sleep(self.interval)

    def start(self):
        self.baseline_bytes = self._rss_bytes()
        self.peak_bytes = self.baseline_bytes
        self._stop = False
        self._thread = threading.Thread(target=self._watch, daemon=True)
        self._thread.start()

    def stop(self):
        if self._thread is not None:
            self._stop = True
            self._thread.join()
            self._thread = None

    @property
    def peak_mb(self):
        return (self.peak_bytes or 0) / (1024 ** 2)

    @property
    def baseline_mb(self):
        return (self.baseline_bytes or 0) / (1024 ** 2)

    @property
    def delta_peak_mb(self):
        if self.baseline_bytes is None or self.peak_bytes is None:
            return 0.0
        return (self.peak_bytes - self.baseline_bytes) / (1024 ** 2)


### Scaler

In [5]:
class FeatureScaler:
    """
    Standardization (zero mean, unit variance)
    + Min-Max normalization to [0, 1].
    Fitted only on training data within each fold.
    """
    def __init__(self):
        self.mean_ = None
        self.std_ = None
        self.min_ = None
        self.max_ = None

    def fit(self, X):
        X = np.asarray(X, dtype=np.float64)
        self.mean_ = X.mean(axis=0)
        self.std_ = X.std(axis=0)
        self.std_[self.std_ == 0] = 1.0  
        X_std = (X - self.mean_) / self.std_

        self.min_ = X_std.min(axis=0)
        self.max_ = X_std.max(axis=0)
        same = self.max_ == self.min_
        self.max_[same] = self.min_[same] + 1.0  
        
    def transform(self, X):
        X = np.asarray(X, dtype=np.float64)
        X_std = (X - self.mean_) / self.std_
        X_norm = (X_std - self.min_) / (self.max_ - self.min_)
        return X_norm

    def fit_transform(self, X):
        self.fit(X)
        return self.transform(X)

## Kelas Ekstraksi Fitur

In [6]:
class FeatureExtractor:
    """
    Modular feature pipeline (bisa untuk B1, B2, B3, P1, P2, dan ablation):

    Grup fitur (bisa di-ON/OFF):
    - Time-domain: mean RMS, mean ZCR
    - Basic spectral: spectral centroid, bandwidth, roll-off
    - Min/Max freq: min & max freq > -40 dB
    - MFCC: 13 mean MFCC
    - Stat-MFE: 27 mel band × (9 atau 4) statistik
    - RAW MFE: log-mel 19 mel band × 44 frame = 836-D (untuk P2)
    - Chroma: 12-dim chroma STFT (mean)
    - Doppler/motion: mean IF, mean chirp rate, mean spectral flux
    """

    def __init__(self,
                 target_sr=44100,
                 duration=3.0,
                 frame_size=1024,
                 hop_length=512,
                 snr_db=15.0,
                 use_time=True,
                 use_basic_spectral=True,
                 use_minmaxfreq=True,
                 use_mfcc=True,
                 use_mfe=False,      # Stat-MFE
                 use_raw_mfe=False,  # RAW MFE (836-D)
                 use_chroma=True,
                 use_doppler=True,
                 mfe_mode="full"     # "full" (9 stats) atau "selected" (4 stats)
                 ):
        self.target_sr = target_sr
        self.duration = duration
        self.frame_size = frame_size
        self.hop_length = hop_length
        self.snr_db = snr_db

        # flags modulasi fitur
        self.use_time = use_time
        self.use_basic_spectral = use_basic_spectral
        self.use_minmaxfreq = use_minmaxfreq
        self.use_mfcc = use_mfcc
        self.use_mfe = use_mfe          # Stat-MFE (P1 / B3)
        self.use_raw_mfe = use_raw_mfe  # RAW MFE (P2)
        self.use_chroma = use_chroma
        self.use_doppler = use_doppler
        self.mfe_mode = mfe_mode.lower() if isinstance(mfe_mode, str) else "full"

    # ---------- Preset helper ----------

    @classmethod
    def from_preset(cls, preset_name, **kwargs):
        """
        Helper untuk bikin FeatureExtractor sesuai preset:
        - "B1" / "all": semua fitur ON (baseline B1: All Features, TANPA MFE/RAW MFE)
        - "mfcc_only": hanya MFCC (B2)
        - "spec_no_mfcc": centroid, SBW, roll-off, chroma, min/max freq,
                          + Doppler (IF, chirp, flux) tanpa MFCC, tanpa time.
        - "pmfe"/"Complete MFE": hanya Stat-MFE (full stats 9/band)
        - "Proposed MFE but Selected Stats": hanya Stat-MFE (selected stats 4/band)
        - "raw_mfe" / "p2": RAW MFE 836-D (P2)
        kwargs bisa override parameter default (target_sr, dll.)
        """
        preset = preset_name.lower()

        base = dict(
            target_sr=44100,
            duration=3.0,
            frame_size=1024,
            hop_length=512,
            snr_db=15.0,
            use_time=True,
            use_basic_spectral=True,
            use_minmaxfreq=True,
            use_mfcc=True,
            use_mfe=False,
            use_raw_mfe=False,
            use_chroma=True,
            use_doppler=True,
            mfe_mode="full",
        )

        if preset in ("b1", "b1_all", "all"):
            # B1: All features (time + all spectral + doppler), TANPA MFE/RAW MFE
            base.update(dict(
                use_time=True,
                use_basic_spectral=True,
                use_minmaxfreq=True,
                use_mfcc=True,
                use_mfe=False,
                use_raw_mfe=False,
                use_chroma=True,
                use_doppler=True,
                mfe_mode="full",
            ))

        elif preset in ("b2_mfcc", "mfcc_only"):
            # B2: hanya MFCC (single feature group)
            base.update(dict(
                use_time=False,
                use_basic_spectral=False,
                use_minmaxfreq=False,
                use_mfcc=True,
                use_mfe=False,
                use_raw_mfe=False,
                use_chroma=False,
                use_doppler=False,
                mfe_mode="full",
            ))

        elif preset in ("spec_no_mfcc", "spectral_no_mfcc"):
            # Ablasi: centroid, SBW, roll-off, chroma, min/max freq, IF, chirp, flux
            base.update(dict(
                use_time=False,
                use_basic_spectral=True,
                use_minmaxfreq=True,
                use_mfcc=False,
                use_mfe=False,
                use_raw_mfe=False,
                use_chroma=True,
                use_doppler=True,
                mfe_mode="full",
            ))

        # ---- PMFE FULL: Complete MFE (MFE-only, 9 stats per band) ----
        elif preset in (
            "pmfe",
            "mfe_only",
            "proposed_mfe",
            "complete mfe",
            "complete_mfe",
            "jayakumar but mfe",
        ):
            base.update(dict(
                use_time=False,
                use_basic_spectral=False,
                use_minmaxfreq=False,
                use_mfcc=False,
                use_mfe=True,
                use_raw_mfe=False,
                use_chroma=False,
                use_doppler=False,
                mfe_mode="full",
            ))

        # ---- P1: Proposed MFE but Selected Stats (MFE-only, 4 stats per band) ----
        elif preset in (
            "proposed mfe but selected stats",
            "pmfe_selected",
            "mfe_selected",
        ):
            base.update(dict(
                use_time=False,
                use_basic_spectral=False,
                use_minmaxfreq=False,
                use_mfcc=False,
                use_mfe=True,
                use_raw_mfe=False,
                use_chroma=False,
                use_doppler=False,
                mfe_mode="selected",
            ))

        # ---- P2: RAW MFE ----
        elif preset in ("p2_raw_mfe", "raw_mfe", "p2"):
            # P2: RAW MFE 836-D (log-mel, no stats, flatten)
            base.update(dict(
                use_time=False,
                use_basic_spectral=False,
                use_minmaxfreq=False,
                use_mfcc=False,
                use_mfe=False,
                use_raw_mfe=True,
                use_chroma=False,
                use_doppler=False,
                mfe_mode="full",
            ))

        else:
            raise ValueError(f"Unknown feature preset: {preset_name}")

        # allow override
        base.update(kwargs)
        return cls(**base)

    # ---------- Preprocessing ----------

    def preprocess(self, y, sr):
        # Resample if needed
        if sr != self.target_sr:
            y = librosa.resample(y, orig_sr=sr, target_sr=self.target_sr)

        # Fixed duration
        target_len = int(self.duration * self.target_sr)
        if len(y) > target_len:
            y = y[:target_len]
        elif len(y) < target_len:
            pad_width = target_len - len(y)
            y = np.pad(y, (0, pad_width), mode="constant")

        # Normalize amplitude to [-1, 1]
        max_amp = np.max(np.abs(y)) + 1e-12
        y = y / max_amp
        return y, self.target_sr

    # ---------- Augmentation ----------

    def augment(self, y):
        """
        White noise at target SNR dengan dynamic factor 0.5–1.5.
        Signal dan noise dinormalisasi sebelum dicampur.
        """
        signal_power = np.mean(y ** 2)
        noise_power = signal_power / (10.0 ** (self.snr_db / 10.0))

        white_noise = np.random.normal(0.0, np.sqrt(noise_power), size=len(y))
        dynamic_factor = np.random.uniform(0.5, 1.5, size=len(y))
        noise = white_noise * dynamic_factor

        sig_max = np.max(np.abs(y)) + 1e-12
        noise_max = np.max(np.abs(noise)) + 1e-12
        y_norm = y / sig_max
        noise_norm = noise / noise_max

        y_aug = y_norm + noise_norm
        y_aug = y_aug / (np.max(np.abs(y_aug)) + 1e-12)
        return y_aug

    # ---------- Time-domain ----------

    def extract_time_domain_features(self, y):
        frames = librosa.util.frame(
            y,
            frame_length=self.frame_size,
            hop_length=self.hop_length
        ).T  # (n_frames, frame_size)

        rms = np.sqrt(np.mean(frames ** 2, axis=1))

        sign_changes = np.diff(np.sign(frames), axis=1) != 0
        zcr = sign_changes.sum(axis=1) / (2.0 * frames.shape[1])

        mean_rms = float(np.mean(rms))
        mean_zcr = float(np.mean(zcr))
        return np.array([mean_rms, mean_zcr], dtype=np.float64)

    # ---------- Spectrogram helper ----------

    def _compute_spectrogram(self, y):
        """
        Compute STFT + derived matrices once, supaya bisa dipakai
        oleh beberapa grup fitur tanpa menghitung ulang.
        """
        stft = librosa.stft(
            y,
            n_fft=self.frame_size,
            hop_length=self.hop_length,
            window="hann",
            center=False
        )
        P = np.abs(stft) ** 2  # power spectrogram (n_freqs, n_frames)
        freqs = librosa.fft_frequencies(sr=self.target_sr, n_fft=self.frame_size)
        S_db = librosa.power_to_db(P, ref=np.max)
        return stft, P, freqs, S_db

    # ---------- Basic spectral: centroid, bandwidth, rolloff ----------

    def extract_basic_spectral_features(self, P, freqs):
        magnitude_sum = P.sum(axis=0) + 1e-12
        centroid = (freqs[:, None] * P).sum(axis=0) / magnitude_sum

        diff = freqs[:, None] - centroid[None, :]
        variance = (diff ** 2 * P).sum(axis=0) / magnitude_sum
        bandwidth = np.sqrt(variance)

        mean_centroid = float(np.mean(centroid))
        mean_bandwidth = float(np.mean(bandwidth))

        rolloff = librosa.feature.spectral_rolloff(
            S=P,
            sr=self.target_sr,
            roll_percent=0.85
        )[0]
        mean_rolloff = float(np.mean(rolloff))

        return np.array(
            [mean_centroid, mean_bandwidth, mean_rolloff],
            dtype=np.float64
        )

    # ---------- Min / Max frequency > -40 dB ----------

    def extract_minmaxfreq_features(self, S_db, freqs):
        significant = S_db > -40
        min_freqs = []
        max_freqs = []
        for t in range(significant.shape[1]):
            idx = np.where(significant[:, t])[0]
            if idx.size > 0:
                min_freqs.append(freqs[idx[0]])
                max_freqs.append(freqs[idx[-1]])
            else:
                min_freqs.append(0.0)
                max_freqs.append(0.0)
        mean_min_freq = float(np.mean(min_freqs))
        mean_max_freq = float(np.mean(max_freqs))
        return np.array([mean_min_freq, mean_max_freq], dtype=np.float64)

    # ---------- MFCC ----------

    def extract_mfcc_features(self, P):
        mfcc = librosa.feature.mfcc(
            S=librosa.power_to_db(P),
            sr=self.target_sr,
            n_mfcc=13
        )
        mean_mfcc = mfcc.mean(axis=1)  # (13,)
        return mean_mfcc.astype(np.float64)

    # ---------- Proposed Stat-MFE (Mel Filterbank Energy) ----------

    def extract_mfe_features(self, P):
        """
        Statistical Mel-Filterbank Energy (Stat-MFE):
        - Hitung mel-spectrogram dari power spectrogram P
        - Konversi ke dB (log-mel)
        - Kalau self.mfe_mode == "full":
            9 statistik/band: [mean, std, min, max, median, p10, p90, p25, p75]
          else (selected):
            4 statistik/band: [mean, min, max, median]
        """
        n_mels = 27

        mel_spec = librosa.feature.melspectrogram(
            S=P,
            sr=self.target_sr,
            n_fft=self.frame_size,
            hop_length=self.hop_length,
            n_mels=n_mels
        )  # (27, n_frames)

        mel_db = librosa.power_to_db(mel_spec, ref=np.max)  # (27, n_frames)

        features_per_band = []
        use_full = (self.mfe_mode == "full")

        for b in range(n_mels):
            band_vals = mel_db[b, :]

            mean_val   = float(np.mean(band_vals))
            std_val    = float(np.std(band_vals))
            min_val    = float(np.min(band_vals))
            max_val    = float(np.max(band_vals))
            median_val = float(np.median(band_vals))

            if use_full:
                p10 = float(np.percentile(band_vals, 10))
                p90 = float(np.percentile(band_vals, 90))
                p25 = float(np.percentile(band_vals, 25))
                p75 = float(np.percentile(band_vals, 75))

                features_per_band.append([
                    mean_val, std_val, min_val, max_val,
                    median_val, p10, p90, p25, p75
                ])
            else:
                # Selected stats: mean, min, max, median
                features_per_band.append([
                    mean_val, min_val, max_val, median_val
                ])

        stat_mfe = np.asarray(features_per_band, dtype=np.float64).reshape(-1)
        return stat_mfe

    # ---------- RAW MFE (P2) ----------

    def extract_raw_mfe_features(self, P):
        """
        RAW Mel-Filterbank Energy untuk P2:
        - Hitung mel-spectrogram dari power spectrogram P
        - Konversi ke dB (log-mel)
        - Normalisasi ke representasi fixed-size:
          19 mel band × 44 frame = 836-D (flatten)
        """
        n_mels = 19
        target_frames = 44  # 19 * 44 = 836

        mel_spec = librosa.feature.melspectrogram(
            S=P,
            sr=self.target_sr,
            n_fft=self.frame_size,
            hop_length=self.hop_length,
            n_mels=n_mels
        )  # (19, T)

        mel_db = librosa.power_to_db(mel_spec, ref=np.max)  # (19, T)
        n_bands, T = mel_db.shape

        if T > target_frames:
            # downsample time axis dengan memilih index secara merata
            idx = np.linspace(0, T - 1, num=target_frames).astype(int)
            mel_db_fixed = mel_db[:, idx]
        elif T < target_frames:
            # pad di ujung kanan dengan nilai minimum per band
            pad_width = target_frames - T
            pad_vals = np.min(mel_db, axis=1, keepdims=True)
            pad = np.repeat(pad_vals, pad_width, axis=1)
            mel_db_fixed = np.concatenate([mel_db, pad], axis=1)
        else:
            mel_db_fixed = mel_db

        raw_mfe = mel_db_fixed.reshape(-1).astype(np.float64)  # (836,)
        return raw_mfe

    # ---------- Chroma ----------

    def extract_chroma_features(self, stft):
        chroma = librosa.feature.chroma_stft(
            S=np.abs(stft),
            sr=self.target_sr,
            n_fft=self.frame_size,
            hop_length=self.hop_length
        )
        mean_chroma = chroma.mean(axis=1)  # (12,)
        return mean_chroma.astype(np.float64)

    # ---------- Doppler / motion ----------

    def extract_doppler_features(self, y):
        analytic = hilbert(y)
        phase = np.unwrap(np.angle(analytic))
        dt = 1.0 / self.target_sr
        inst_freq = np.diff(phase) / (2.0 * np.pi * dt)
        mean_if = float(np.mean(inst_freq))

        chirp_rate = np.diff(inst_freq) / dt
        mean_chirp = float(np.mean(chirp_rate))

        stft_mag = np.abs(librosa.stft(
            y,
            n_fft=self.frame_size,
            hop_length=self.hop_length,
            window="hann",
            center=False
        ))
        norm = stft_mag.sum(axis=0, keepdims=True) + 1e-12
        S_norm = stft_mag / norm

        flux_values = []
        prev = np.zeros(S_norm.shape[0])
        for t in range(S_norm.shape[1]):
            cur = S_norm[:, t]
            flux = np.sum(np.maximum(cur - prev, 0.0))
            flux_values.append(flux)
            prev = cur
        mean_flux = float(np.mean(flux_values))

        return np.array([mean_if, mean_chirp, mean_flux], dtype=np.float64)

    # ---------- Public API ----------

    def extract_features_from_signal(self, y, sr, apply_augmentation=False):
        y_proc, sr_proc = self.preprocess(y, sr)
        if apply_augmentation:
            y_proc = self.augment(y_proc)

        features = []

        # Time-domain
        if self.use_time:
            features.append(self.extract_time_domain_features(y_proc))

        # Spectral-domain (pakai STFT shared kalau perlu)
        need_spectral = any([
            self.use_basic_spectral,
            self.use_minmaxfreq,
            self.use_mfcc,
            self.use_mfe,
            self.use_raw_mfe,
            self.use_chroma,
        ])

        stft = P = freqs = S_db = None
        if need_spectral:
            stft, P, freqs, S_db = self._compute_spectrogram(y_proc)

        # Urutan: [basic spectral, min/max, MFCC, MFE, RAW MFE, chroma]
        if self.use_basic_spectral and P is not None:
            features.append(self.extract_basic_spectral_features(P, freqs))

        if self.use_minmaxfreq and S_db is not None:
            features.append(self.extract_minmaxfreq_features(S_db, freqs))

        if self.use_mfcc and P is not None:
            features.append(self.extract_mfcc_features(P))

        if self.use_mfe and P is not None:
            features.append(self.extract_mfe_features(P))

        if self.use_raw_mfe and P is not None:
            features.append(self.extract_raw_mfe_features(P))

        if self.use_chroma and stft is not None:
            features.append(self.extract_chroma_features(stft))

        # Doppler / motion
        if self.use_doppler:
            features.append(self.extract_doppler_features(y_proc))

        if not features:
            raise ValueError("No feature groups enabled in FeatureExtractor.")

        full_feat = np.concatenate(features, axis=0)
        return full_feat

    def extract_features_from_file(self, filepath, label=None, augment_for_labels=None):
        y, sr = librosa.load(filepath, sr=None, mono=True)

        feats = []
        # Original
        feats.append(self.extract_features_from_signal(y, sr, apply_augmentation=False))

        # Augmented (only for some labels)
        if augment_for_labels is not None and label in augment_for_labels:
            feats.append(self.extract_features_from_signal(y, sr, apply_augmentation=True))

        return feats

## Akuisisi Folder dan Audio

In [7]:
class AudioDataset:
    """
    Folder structure:
    base_dir/
        ambulance/*.wav
        firetruck/*.wav
        traffic/*.wav

    Menghasilkan X (fitur) dan y (label string),
    dengan augmentasi untuk ambulance & firetruck (default).
    """
    def __init__(self, base_dir, feature_extractor,
                 class_names=None,
                 augment=True,
                 augment_for=("ambulance", "firetruck")):
        self.base_dir = base_dir
        self.fe = feature_extractor
        self.augment = augment
        if class_names is None:
            self.class_names = ["ambulance", "firetruck", "traffic"]
        else:
            self.class_names = class_names
        self.augment_for = set(augment_for)

    def load(self):
        X = []
        y = []

        for label in self.class_names:
            class_dir = os.path.join(self.base_dir, label)
            if not os.path.isdir(class_dir):
                continue
            for fname in sorted(os.listdir(class_dir)):
                if not fname.lower().endswith(".wav"):
                    continue
                fpath = os.path.join(class_dir, fname)
                features_list = self.fe.extract_features_from_file(
                    fpath,
                    label=label,
                    augment_for_labels=self.augment_for if self.augment else None
                )
                for feat in features_list:
                    X.append(feat)
                    y.append(label)

        X = np.asarray(X, dtype=np.float64)
        y = np.asarray(y)
        return X, y

## Model-model (Machine Learning)

### Base Model

In [8]:
def create_base_models():
    """Base classifiers untuk B1/B2/B3/P1/P2."""
    models = {
        "SVM": SVC(
            kernel="rbf",
            C=10.0,
            gamma="scale",
            probability=True,
            random_state=SEED
        ),
        "KNN": KNeighborsClassifier(
            n_neighbors=3,
            weights="distance"
        ),
        "RandomForest": RandomForestClassifier(
            n_estimators=200,
            max_depth=None,
            min_samples_split=2,
            random_state=SEED,
            n_jobs=-1
        ),
        "AdaBoost": AdaBoostClassifier(
            n_estimators=100,
            learning_rate=1.0,
            random_state=SEED
        )
    }
    return models

### Ensembel Model

In [9]:
class StackedEnsemble:
    """
    Stacked ensemble:
    - Base: SVM, KNN, RF, AdaBoost (probability outputs)
    - Meta-learner: SVM (rbf)
    """
    def __init__(self, base_models, meta_learner=None):
        self.base_models = base_models
        if meta_learner is None:
            self.meta_learner = SVC(
                kernel="rbf",
                C=10.0,
                gamma="scale",
                probability=False,
                random_state=SEED
            )
        else:
            self.meta_learner = meta_learner

    def fit(self, X_train, y_train):
        X_base, X_meta, y_base, y_meta = train_test_split(
            X_train, y_train,
            test_size=0.2,
            stratify=y_train,
            random_state=SEED
        )

        for model in self.base_models.values():
            model.fit(X_base, y_base)

        meta_features = []
        for name, model in self.base_models.items():
            proba = model.predict_proba(X_meta)
            meta_features.append(proba)
        meta_X = np.hstack(meta_features)

        self.meta_learner.fit(meta_X, y_meta)

    def predict(self, X):
        meta_features = []
        for name, model in self.base_models.items():
            proba = model.predict_proba(X)
            meta_features.append(proba)
        meta_X = np.hstack(meta_features)
        y_pred = self.meta_learner.predict(meta_X)
        return y_pred

## Runner

In [10]:
# =========================
# Experiment runner
# =========================

class ExperimentRunner:
    def __init__(self, data_dir, feature_preset="B1"):
        self.data_dir = data_dir
        self.feature_preset = feature_preset
        self.fe = FeatureExtractor.from_preset(feature_preset)
        self.dataset = AudioDataset(data_dir, self.fe)
        self.label_encoder = LabelEncoder()

    def run(self, n_splits=5):
        # 1. Feature extraction timing (seluruh dataset, termasuk augment)
        feat_timer = Timer()
        feat_mem = MemoryTracker(interval=0.02)

        feat_mem.start()
        feat_timer.start()
        X_raw, y_labels = self.dataset.load()
        feat_timer.stop()
        feat_mem.stop()

        feat_mem_usage = feat_mem.delta_peak_mb  # ✅ peak delta MB


        n_samples_total = X_raw.shape[0]
        feat_time_total = feat_timer.get_elapsed()
        feat_time_per_sample = feat_time_total / n_samples_total

        print(f"✅ Dataset loaded and features extracted (preset: {self.feature_preset}).")
        print(f"   Total samples (incl. augmentation): {n_samples_total}")
        print(f"   Feature dimension: {X_raw.shape[1]}")
        print(f"   Total feature extraction time: {feat_time_total:.3f} s")
        print(f"   Avg feature extraction time per sample: {feat_time_per_sample:.6f} s")
        print(f"   Extra memory during feature extraction: {feat_mem_usage:.2f} MB")

        # Encode labels (0,1,2)
        y = self.label_encoder.fit_transform(y_labels)

        # 2. Cross-validation setup
        skf = StratifiedKFold(
            n_splits=n_splits,
            shuffle=True,
            random_state=SEED
        )

        model_names = ["SVM", "KNN", "RandomForest", "AdaBoost", "StackedEnsemble"]
        acc_logs = {name: [] for name in model_names}
        f1_logs = {name: [] for name in model_names}
        infer_time_logs = {name: [] for name in model_names}  # per-sample on test set
        mem_logs = {name: [] for name in model_names}         # train+test peak

        fold_idx = 1
        for train_idx, test_idx in skf.split(X_raw, y):
            print("\n" + "=" * 60)
            print(f"🔍 FOLD {fold_idx}/{n_splits}")
            print("=" * 60)

            X_train_raw, X_test_raw = X_raw[train_idx], X_raw[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            # Scaling (fit only on training fold)
            scaler = FeatureScaler()
            X_train = scaler.fit_transform(X_train_raw)
            X_test = scaler.transform(X_test_raw)

            # --- Base models ---
            base_models = create_base_models()

            for name, model in base_models.items():
                print(f"\n▶ Training {name}...")

                mem_tracker = MemoryTracker(interval=0.02)
                train_timer = Timer()
                infer_timer = Timer()

                mem_tracker.start()

                # Train
                train_timer.start()
                model.fit(X_train, y_train)
                train_timer.stop()

                # Inference on test set
                infer_timer.start()
                y_pred = model.predict(X_test)
                infer_timer.stop()

                mem_tracker.stop()

                
                print(f"   {name} baseline RSS: {mem_tracker.baseline_mb:.2f} MB")
                print(f"   {name} peak RSS: {mem_tracker.peak_mb:.2f} MB")


                mem_usage = mem_tracker.peak_mb   # ✅ PEAK RAM (RSS) selama train+test block  # ✅ peak delta MB

                infer_time_total = infer_timer.get_elapsed()
                infer_time_per_sample = infer_time_total / len(X_test)

                acc = accuracy_score(y_test, y_pred)
                f1 = f1_score(y_test, y_pred, average="macro")

                acc_logs[name].append(acc)
                f1_logs[name].append(f1)
                infer_time_logs[name].append(infer_time_per_sample)
                mem_logs[name].append(mem_usage)

                print(f"   {name} fold accuracy: {acc:.4f}, F1-macro: {f1:.4f}")
                print(f"   {name} inference time per sample (test): {infer_time_per_sample:.6f} s")
                print(f"   {name} peak RAM (RSS) train+test): {mem_usage:.2f} MB")
                del y_pred
                gc.collect()

            # --- Stacked ensemble ---
            print("\n▶ Training Stacked Ensemble (base: SVM, KNN, RF, AdaBoost; meta: SVM)...")
            ensemble_base_models = create_base_models()
            ensemble = StackedEnsemble(ensemble_base_models)

            mem_tracker = MemoryTracker(interval=0.02)
            train_timer = Timer()
            infer_timer = Timer()

            mem_tracker.start()

            train_timer.start()
            ensemble.fit(X_train, y_train)
            train_timer.stop()

            infer_timer.start()
            y_pred_ens = ensemble.predict(X_test)
            infer_timer.stop()

            mem_tracker.stop()
            name = "StackedEnsemble"  

            print(f"   {name} baseline RSS: {mem_tracker.baseline_mb:.2f} MB")
            print(f"   {name} peak RSS: {mem_tracker.peak_mb:.2f} MB")


            mem_usage = mem_tracker.peak_mb   # ✅ PEAK RAM (RSS) selama train+test block

            infer_time_total = infer_timer.get_elapsed()
            infer_time_per_sample = infer_time_total / len(X_test)

            acc = accuracy_score(y_test, y_pred_ens)
            f1 = f1_score(y_test, y_pred_ens, average="macro")

            name = "StackedEnsemble"
            acc_logs[name].append(acc)
            f1_logs[name].append(f1)
            infer_time_logs[name].append(infer_time_per_sample)
            mem_logs[name].append(mem_usage)

            print(f"   Stacked Ensemble fold accuracy: {acc:.4f}, F1-macro: {f1:.4f}")
            print(f"   Stacked Ensemble inference time per sample (test): {infer_time_per_sample:.6f} s")
            print(f"   Stacked Ensemble peak RAM (RSS) train+test): {mem_usage:.2f} MB")

            del y_pred_ens
            gc.collect()

            fold_idx += 1

        # 3. Summarize across folds
        summary_rows = []
        for name in model_names:
            acc_mean = np.mean(acc_logs[name])
            acc_std = np.std(acc_logs[name])
            f1_mean = np.mean(f1_logs[name])
            f1_std = np.std(f1_logs[name])
            infer_mean = np.mean(infer_time_logs[name])
            mem_mean = np.mean(mem_logs[name])

            summary_rows.append({
                "Model": name,
                "Accuracy (mean±std)": f"{acc_mean:.4f} ± {acc_std:.4f}",
                "F1 Score (mean±std)": f"{f1_mean:.4f} ± {f1_std:.4f}",
                "Run Time Ekstraksi Fitur per Sampel [s]": f"{feat_time_per_sample:.6f}",
                "Run Time Inference per Sampel [s] (mean test)": f"{infer_mean:.6f}",
                "Memory usage tambahan [MB] (train+test, mean)": f"{mem_mean:.2f}"
            })

        results_df = pd.DataFrame(summary_rows)
        results_df.insert(0, "No", np.arange(1, len(results_df) + 1))

        print("\n" + "=" * 80)
        print("HASIL 5-FOLD CROSS VALIDATION")
        print(f"(Preset fitur: {self.feature_preset})")
        print("=" * 80)
        print(results_df.to_string(index=False))

        return results_df

## Main Runner

In [11]:
if __name__ == "__main__":
    data_dir = "./sounds"

    # B1: All features (baseline Jayakumar-style, tanpa MFE/RAW MFE)
    runner_b1 = ExperimentRunner(data_dir, feature_preset="B1")
    results_b1 = runner_b1.run(n_splits=5)

    # B2: MFCC-only
    runner_b2 = ExperimentRunner(data_dir, feature_preset="mfcc_only")
    results_b2 = runner_b2.run(n_splits=5)

    # Ablasi: centroid, SBW, roll-off, chroma, min/max freq, IF, chirp, flux
    runner_spec = ExperimentRunner(data_dir, feature_preset="spec_no_mfcc")
    results_spec = runner_spec.run(n_splits=5)

    # B3 / P1 (PMFE FULL): Statistical MFE-only (9 stats/band, MFE vs MFCC head-to-head)
    runner_pmfe = ExperimentRunner(data_dir, feature_preset="Complete MFE")
    results_pmfe = runner_pmfe.run(n_splits=5)

    # P1: Proposed MFE with selected stats (mean, min, max, median)
    runner_p1 = ExperimentRunner(data_dir, feature_preset="Proposed MFE but Selected Stats")
    results_p1 = runner_p1.run(n_splits=5)

    # P2: RAW MFE 836-D (Raw-MFE Ensemble, 4 base + meta-SVM, pakai pipeline yang sama)
    runner_p2 = ExperimentRunner(data_dir, feature_preset="raw_mfe")
    results_p2 = runner_p2.run(n_splits=5)

✅ Dataset loaded and features extracted (preset: B1).
   Total samples (incl. augmentation): 1000
   Feature dimension: 35
   Total feature extraction time: 45.791 s
   Avg feature extraction time per sample: 0.045791 s
   Extra memory during feature extraction: 108.35 MB

🔍 FOLD 1/5

▶ Training SVM...
   SVM baseline RSS: 468.55 MB
   SVM peak RSS: 468.67 MB
   SVM fold accuracy: 0.9450, F1-macro: 0.9542
   SVM inference time per sample (test): 0.000015 s
   SVM peak RAM (RSS) train+test): 468.67 MB

▶ Training KNN...
   KNN baseline RSS: 468.71 MB
   KNN peak RSS: 469.64 MB
   KNN fold accuracy: 0.9150, F1-macro: 0.9289
   KNN inference time per sample (test): 0.010153 s
   KNN peak RAM (RSS) train+test): 469.64 MB

▶ Training RandomForest...
   RandomForest baseline RSS: 469.65 MB
   RandomForest peak RSS: 470.65 MB
   RandomForest fold accuracy: 0.9350, F1-macro: 0.9458
   RandomForest inference time per sample (test): 0.000186 s
   RandomForest peak RAM (RSS) train+test): 470.65 M

In [12]:
if __name__ == "__main__":
    data_dir = "./sounds"

    # B1: All features (baseline Jayakumar-style, tanpa MFE/RAW MFE)
    runner_b1 = ExperimentRunner(data_dir, feature_preset="B1")
    results_b1 = runner_b1.run(n_splits=5)

    # B2: MFCC-only
    runner_b2 = ExperimentRunner(data_dir, feature_preset="mfcc_only")
    results_b2 = runner_b2.run(n_splits=5)

    # Ablasi: centroid, SBW, roll-off, chroma, min/max freq, IF, chirp, flux
    runner_spec = ExperimentRunner(data_dir, feature_preset="spec_no_mfcc")
    results_spec = runner_spec.run(n_splits=5)

    # B3 / P1 (PMFE FULL): Statistical MFE-only (9 stats/band, MFE vs MFCC head-to-head)
    runner_pmfe = ExperimentRunner(data_dir, feature_preset="Complete MFE")
    results_pmfe = runner_pmfe.run(n_splits=5)

    # P1: Proposed MFE with selected stats (mean, min, max, median)
    runner_p1 = ExperimentRunner(data_dir, feature_preset="Proposed MFE but Selected Stats")
    results_p1 = runner_p1.run(n_splits=5)

    # P2: RAW MFE 836-D (Raw-MFE Ensemble, 4 base + meta-SVM, pakai pipeline yang sama)
    runner_p2 = ExperimentRunner(data_dir, feature_preset="raw_mfe")
    results_p2 = runner_p2.run(n_splits=5)

✅ Dataset loaded and features extracted (preset: B1).
   Total samples (incl. augmentation): 1000
   Feature dimension: 35
   Total feature extraction time: 33.012 s
   Avg feature extraction time per sample: 0.033012 s
   Extra memory during feature extraction: 14.82 MB

🔍 FOLD 1/5

▶ Training SVM...
   SVM baseline RSS: 479.14 MB
   SVM peak RSS: 479.16 MB
   SVM fold accuracy: 0.9550, F1-macro: 0.9625
   SVM inference time per sample (test): 0.000014 s
   SVM peak RAM (RSS) train+test): 479.16 MB

▶ Training KNN...
   KNN baseline RSS: 479.14 MB
   KNN peak RSS: 479.16 MB
   KNN fold accuracy: 0.9000, F1-macro: 0.9163
   KNN inference time per sample (test): 0.000042 s
   KNN peak RAM (RSS) train+test): 479.16 MB

▶ Training RandomForest...
   RandomForest baseline RSS: 479.12 MB
   RandomForest peak RSS: 479.76 MB
   RandomForest fold accuracy: 0.9400, F1-macro: 0.9500
   RandomForest inference time per sample (test): 0.000261 s
   RandomForest peak RAM (RSS) train+test): 479.76 MB

In [13]:
if __name__ == "__main__":
    data_dir = "./sounds"

    # B1: All features (baseline Jayakumar-style, tanpa MFE/RAW MFE)
    runner_b1 = ExperimentRunner(data_dir, feature_preset="B1")
    results_b1 = runner_b1.run(n_splits=5)

    # B2: MFCC-only
    runner_b2 = ExperimentRunner(data_dir, feature_preset="mfcc_only")
    results_b2 = runner_b2.run(n_splits=5)

    # Ablasi: centroid, SBW, roll-off, chroma, min/max freq, IF, chirp, flux
    runner_spec = ExperimentRunner(data_dir, feature_preset="spec_no_mfcc")
    results_spec = runner_spec.run(n_splits=5)

    # B3 / P1 (PMFE FULL): Statistical MFE-only (9 stats/band, MFE vs MFCC head-to-head)
    runner_pmfe = ExperimentRunner(data_dir, feature_preset="Complete MFE")
    results_pmfe = runner_pmfe.run(n_splits=5)

    # P1: Proposed MFE with selected stats (mean, min, max, median)
    runner_p1 = ExperimentRunner(data_dir, feature_preset="Proposed MFE but Selected Stats")
    results_p1 = runner_p1.run(n_splits=5)

    # P2: RAW MFE 836-D (Raw-MFE Ensemble, 4 base + meta-SVM, pakai pipeline yang sama)
    runner_p2 = ExperimentRunner(data_dir, feature_preset="raw_mfe")
    results_p2 = runner_p2.run(n_splits=5)

✅ Dataset loaded and features extracted (preset: B1).
   Total samples (incl. augmentation): 1000
   Feature dimension: 35
   Total feature extraction time: 34.885 s
   Avg feature extraction time per sample: 0.034885 s
   Extra memory during feature extraction: 15.68 MB

🔍 FOLD 1/5

▶ Training SVM...
   SVM baseline RSS: 482.36 MB
   SVM peak RSS: 482.38 MB
   SVM fold accuracy: 0.9550, F1-macro: 0.9625
   SVM inference time per sample (test): 0.000016 s
   SVM peak RAM (RSS) train+test): 482.38 MB

▶ Training KNN...
   KNN baseline RSS: 482.36 MB
   KNN peak RSS: 482.38 MB
   KNN fold accuracy: 0.8800, F1-macro: 0.8992
   KNN inference time per sample (test): 0.000047 s
   KNN peak RAM (RSS) train+test): 482.38 MB

▶ Training RandomForest...
   RandomForest baseline RSS: 482.36 MB
   RandomForest peak RSS: 483.01 MB
   RandomForest fold accuracy: 0.9400, F1-macro: 0.9500
   RandomForest inference time per sample (test): 0.000326 s
   RandomForest peak RAM (RSS) train+test): 483.01 MB